In [1]:
import os.path
import pandas as pd

In [2]:
if not os.path.isfile("testing-data.csv"):
    raise Exception("Missing CSV File", "please make sure to have the file 'testing-data.csv' in this directory.")

## Data Cleanup

In [3]:
def display_df_info(df):
    """
    Display the info for the dataframe
    """
    # iterate thru the columns and display info
    for column in df.columns:
        print(f"{column} ({df[column].dtype})")
        if (df[column].dtype == object):
            unique = df[column].unique().tolist()
            print(len(unique))
            # ignore the really long lists
            if (len(unique) < 100):
                print(unique)
            else: print("...")
        else:
            print(df[column].describe())
        print("\n" + "-" * 10)

In [4]:
def remove_outlier(df, col_name):
    """
    remove the outliers of the data using the interquartile range.
    """
    # get the first interquartile range
    q1 = df[col_name].quantile(0.25)
    # get the last interquartile range
    q3 = df[col_name].quantile(0.75)
    # Interquartile range
    iqr = q3 - q1
    # grab the lowwer and upper bounds
    bounds_low  = q1 - 1.5 * iqr
    bounds_high = q3 + 1.5 * iqr
    # apply and send off
    return df.loc[(df[col_name] > bounds_low) & (df[col_name] < bounds_high)]

In [5]:
def format_cylinders_column(cylinders):
    """
    convert the cylinders column from ['10 cylinders', '8 cylinders', ...] to just the values
    """
    # if the number is a float, 
    # it is probably a not a number
    if (type(cylinders) == float):
        return ""
    # if the count other, return zero
    if (cylinders == "other"):
        return "0"
    #
    return cylinders.replace(" cylinders", "")

In [6]:
def clean_used_cars_data(df):
    # remove the cars with zero price
    df = df[df["price"] != 0]
    # remove the cars with one dollar price,
    # looks like there were tons of cars being
    # sold for one dollar, and skews our data.
    # there is a great discussion to be had over if 
    # we want to add price under $100 for instance
    # to the filtering.
    df = df[df["price"] != 1]
    # remove the price outliers, someone put their
    # phone number as the price lol
    df = remove_outlier(df, "price")
    # remove the price outliers, people be putting
    # millions here for cars less than a year old...
    df = remove_outlier(df, "odometer")
    
    # convert to datetime
    df["posting_date"] = pd.to_datetime(df["posting_date"], infer_datetime_format=True) 
    
    # format the string to numbers 
    df["cylinders"] = df["cylinders"].apply(format_cylinders_column)
    df["cylinders"] = pd.to_numeric(df["cylinders"], errors='coerce')
    
    # remove the salvage cars
    df = df[df["condition"] != "salvage"]
    # remove the titles salvage cars, parts only
    df = df[df["title_status"] != "salvage"]
    df = df[df["title_status"] != "parts only"] 
    # thinking this is a good idea to keep in
    #df = df[df["title_status"] != "missing"]
    
    return df.reset_index(drop=True)

In [7]:
# Read the raw data
raw_df = pd.read_csv("testing-data.csv") 
raw_df

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,...,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,...,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,...,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,...,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


In [8]:
# Select the features to train the data
target_feature = "price"

# no nulls for the selected features
selected_features = [
    "id",
#     "url",
    "region",
#     "region_url",
    "price",
    "year",
    "manufacturer",
    "model",
    "condition",
    "cylinders",
    "fuel",
    "odometer",
    "title_status",
    "transmission",
#     "VIN",
    "drive",
    "size",
    "type",
    "paint_color",
#     "image_url",
#     "description",
    "county",
    "state",
    "lat",
    "long",
    "posting_date",
]

# just in case we remove the selected feature in testing
if target_feature not in selected_features:
    selected_features.append(target_feature)

# select the features we need, and rop the null columns and null rows
training_df_with_nulls = clean_used_cars_data(raw_df)[selected_features]
training_df_with_nulls.to_csv("clean-testing-data-with-nulls.csv", index=False)
training_df_with_nulls

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,...,transmission,drive,size,type,paint_color,county,state,lat,long,posting_date
0,7316814884,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8.0,gas,57923.0,...,other,NaN,NaN,pickup,white,NaN,al,32.590000,-85.480000,2021-05-04 12:31:18-05:00
1,7316814758,auburn,22590,2010.0,chevrolet,silverado 1500,good,8.0,gas,71229.0,...,other,NaN,NaN,pickup,blue,NaN,al,32.590000,-85.480000,2021-05-04 12:31:08-05:00
2,7316814989,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8.0,gas,19160.0,...,other,NaN,NaN,pickup,red,NaN,al,32.590000,-85.480000,2021-05-04 12:31:25-05:00
3,7316743432,auburn,30990,2017.0,toyota,tundra double cab sr,good,8.0,gas,41124.0,...,other,NaN,NaN,pickup,red,NaN,al,32.590000,-85.480000,2021-05-04 10:41:31-05:00
4,7316356412,auburn,15000,2013.0,ford,f-150 xlt,excellent,6.0,gas,128000.0,...,automatic,rwd,full-size,truck,black,NaN,al,32.592000,-85.518900,2021-05-03 14:02:03-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374273,7301591192,wyoming,23590,2019.0,nissan,maxima s sedan 4d,good,6.0,gas,32226.0,...,other,fwd,NaN,sedan,NaN,NaN,wy,33.786500,-84.445400,2021-04-04 03:21:31-06:00
374274,7301591187,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,...,other,fwd,NaN,sedan,red,NaN,wy,33.786500,-84.445400,2021-04-04 03:21:29-06:00
374275,7301591147,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,...,other,NaN,NaN,hatchback,white,NaN,wy,33.779214,-84.411811,2021-04-04 03:21:17-06:00
374276,7301591140,wyoming,28990,2018.0,lexus,es 350 sedan 4d,good,6.0,gas,30112.0,...,other,fwd,NaN,sedan,silver,NaN,wy,33.786500,-84.445400,2021-04-04 03:21:11-06:00


In [10]:
# drop all the null columns and values
training_df = training_df_with_nulls.dropna(axis="columns", how="all").dropna().reset_index(drop=True)
training_df.to_csv("clean-testing-data.csv", index=False)
training_df

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,lat,long,posting_date
0,7316356412,auburn,15000,2013.0,ford,f-150 xlt,excellent,6.0,gas,128000.0,clean,automatic,rwd,full-size,truck,black,al,32.592000,-85.518900,2021-05-03 14:02:03-05:00
1,7314560853,auburn,19900,2004.0,ford,f250 super duty,good,8.0,diesel,88000.0,clean,automatic,4wd,full-size,pickup,blue,al,32.547500,-85.468200,2021-04-29 17:19:18-05:00
2,7313406529,auburn,14000,2012.0,honda,odyssey,excellent,6.0,gas,95000.0,clean,automatic,fwd,full-size,mini-van,silver,al,32.628739,-85.461820,2021-04-27 12:20:01-05:00
3,7312847466,auburn,22500,2001.0,ford,f450,good,8.0,diesel,144700.0,clean,manual,rwd,full-size,truck,white,al,32.630400,-85.401600,2021-04-26 11:15:36-05:00
4,7312144944,auburn,15000,2017.0,dodge,charger rt 4dr sedan,excellent,8.0,gas,90000.0,rebuilt,automatic,rwd,mid-size,sedan,grey,al,32.822400,-85.770400,2021-04-24 18:39:59-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71901,7303112347,wyoming,15999,2018.0,chevrolet,"cruze, lt",excellent,4.0,gas,36465.0,clean,automatic,fwd,mid-size,sedan,black,wy,41.138284,-104.784799,2021-04-07 09:03:17-06:00
71902,7302963673,wyoming,18000,2005.0,chevrolet,silverado 1500 lt 4x4,excellent,8.0,gas,130000.0,lien,automatic,4wd,full-size,truck,blue,wy,43.452800,-110.739300,2021-04-06 21:04:03-06:00
71903,7302963273,wyoming,18000,1990.0,jeep,gand wagoneer,good,8.0,gas,114400.0,clean,automatic,4wd,full-size,SUV,black,wy,43.452800,-110.739300,2021-04-06 21:02:26-06:00
71904,7302384818,wyoming,9800,1985.0,nissan,300zx coupe with t-tops,like new,6.0,gas,115000.0,clean,automatic,rwd,sub-compact,hatchback,red,wy,41.143700,-104.796200,2021-04-05 18:10:52-06:00


In [11]:
display_df_info(training_df)

id (int64)
count    7.190600e+04
mean     7.311841e+09
std      4.269575e+06
min      7.301584e+09
25%      7.308770e+09
50%      7.313071e+09
75%      7.315399e+09
max      7.317101e+09
Name: id, dtype: float64

----------
region (object)
403
...

----------
price (int64)
count    71906.000000
mean     13462.457611
std      10923.381867
min          3.000000
25%       5500.000000
50%       9950.000000
75%      17995.000000
max      58500.000000
Name: price, dtype: float64

----------
year (float64)
count    71906.000000
mean      2008.280283
std          9.869512
min       1900.000000
25%       2005.000000
50%       2010.000000
75%       2014.000000
max       2022.000000
Name: year, dtype: float64

----------
manufacturer (object)
41
['ford', 'honda', 'dodge', 'chrysler', 'toyota', 'jeep', 'lexus', 'chevrolet', 'bmw', 'gmc', 'mercedes-benz', 'mazda', 'ram', 'nissan', 'audi', 'mitsubishi', 'infiniti', 'volkswagen', 'kia', 'hyundai', 'fiat', 'acura', 'cadillac', 'lincoln', 'jaguar', 'sa